In [20]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from lxml import  html 
import requests

In [21]:
xingzhengqu_cn = ['浦东','黄浦']
xingzhengqu = ['pudong','huangpu']
url = 'https://sh.lianjia.com/zufang/huangpu/'

In [25]:
URL = []
def get_html(url,page):
#     html = requests.get(url+'/pg%s'%str(i)).text
    url_n = url+'/pg%s'%str(page)
    html = requests.get(url_n).text
    table = BeautifulSoup(html,'lxml').find('div',{'class':'con-box'}).find_all('li',{'data-el':'zufang'})
    
    table2 = BeautifulSoup(html,'lxml')
#     使用正则表达式获得最大页码数
    pattern  = re.compile('"totalPage":(.*?),".*?"')
    lastpage = re.findall(pattern,str(table2))
    
    pattern  = re.compile('"totalPage":(.*?),".*?"')
    
    pattern_mianji = re.compile("(.*?)平米.*?")
    pattern_quyu = re.compile("(.*?)租房")
    pattern_updatetime = re.compile("(.*?)\s更新")
    
    result =[]
    
    for li in table:
        item = dict()
        item[u'小区'] = li.find('div', {'class': 'where'}).find('a').get_text().replace('\xa0', '')
        item[u'房型'] = li.find('div', {'class': 'where'}).find_all('span')[2].get_text().replace('\xa0', '')
        item[u'面积'] = re.findall(pattern_mianji, li.find('div', {'class': 'where'}).find_all('span')[3].get_text())[0]
        item[u'朝向'] = li.find('div', {'class': 'where'}).find_all('span')[4].get_text()
        item[u'地区'] = re.findall(pattern_quyu, li.find('div', {'class': 'other'}).find('a').get_text())[0]
        item[u'价格'] = li.find('div', {'class': 'price'}).find('span', {'class': 'num'}).get_text()
        item[u'楼层'] = li.find('div', {'class': 'con'}).get_text().split('/')[1]
        result.append(item)
    return result

In [28]:
last_page = 36
from collections import defaultdict
data = pd.DataFrame()

In [46]:
for i in range(1,37):
    d = pd.DataFrame(get_html(url,i))
    data = pd.concat([d,data])

In [54]:
data.to_excel('D:\house3.xlsx',index = False,index_label=None)